In [22]:
import datetime
import time

import joblib
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,roc_auc_score, confusion_matrix,classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

import csv

##### Cargar archivo y pipeline

In [15]:
data=pd.read_csv('airline_satisfaction.csv')
pipeline=joblib.load('pipeline_airline.pkl')

In [16]:
# Codificacion variable target
ohe_target=pd.get_dummies(data['satisfaction'],drop_first=True)
data['satisfaction']=ohe_target


##### Crear Funcion Train Model

In [17]:
def train_model():

    # Obtener el tiempo de inicio
    inicio = time.time()

    X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['id', 'satisfaction'], axis=1),# Varaibles X, sin el Id y sin la varaible Y
    data['satisfaction'], # Variable Y
    test_size=0.3, # Tamaño de la muestra 70% Train. 30% Test
    random_state=2023) # Aleatorio seed 2023

    # Fit del Pipeline a las variables X
    pipeline.fit(X_train, X_test)

    # Transformar las variables X segun la ingenieria de caracteristicas
    X_train_trans=pipeline.transform(X_train)
    X_test_trans=pipeline.transform(X_test)

    ## Instanciamos objeto para modelo
    random_forest= RandomForestClassifier()
    random_forest

    # Entrenamiento de modelo
    random_forest.fit(X_train_trans,y_train)

    # Prediccion del Modelo
    y_preds_forest=random_forest.predict(X_test_trans)

    ## Diccionario de hiper parametros
    hyperparam_grid={'n_estimators':[200],
                 'criterion':['gini','entropy','log_loss']
                 }


    grid_search=GridSearchCV(estimator=random_forest,param_grid=hyperparam_grid,scoring=['roc_auc','accuracy'],refit='roc_auc',n_jobs=-1,
                         cv=KFold(n_splits=10,shuffle=True,random_state=2023)) 

    ##Entrenar el modelo
    grid_search.fit(X_train_trans,y_train)

    accuracies=grid_search.cv_results_['mean_test_accuracy']
    roc_aucs=grid_search.cv_results_['mean_test_roc_auc']

    mean_acc =  np.mean([x for x in accuracies if not np.isnan(x)])
    mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

    # Obtener el tiempo de finalización
    fin = time.time()

    # Calcular el tiempo transcurrido
    tiempo_transcurrido = fin - inicio

    # Obtener la fecha y hora actual
    fecha_hora_actual = datetime.datetime.now()

    # Formatear la fecha y hora
    fecha_hora_formateada = fecha_hora_actual.strftime("%Y-%m-%d %H:%M:%S")

    # Crear el contenido a escribir en el archivo
    contenido = f"Fecha y hora de ejecución: {fecha_hora_formateada},El tiempo de ejecución fue de {tiempo_transcurrido} segundos. Mejores HiperParametros del Random Forest:{grid_search.best_params_},Mejor Metrica:{grid_search.best_score_}"

    # Escribir el contenido en un archivo de texto
    with open("modelo.txt", "w") as archivo:
        archivo.write(contenido)


##### Crear Funcion Prediccion Datos

In [23]:
def predict():

    X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['id', 'satisfaction'], axis=1),# Varaibles X, sin el Id y sin la varaible Y
    data['satisfaction'], # Variable Y
    test_size=0.3, # Tamaño de la muestra 70% Train. 30% Test
    random_state=2023) # Aleatorio seed 2023

    # Fit del Pipeline a las variables X
    pipeline.fit(X_train, X_test)

    # Transformar las variables X segun la ingenieria de caracteristicas
    X_train_trans=pipeline.transform(X_train)
    X_test_trans=pipeline.transform(X_test)

    ## Instanciamos objeto para modelo
    random_forest= RandomForestClassifier()
    random_forest

    # Entrenamiento de modelo
    random_forest.fit(X_train_trans,y_train)

    # Prediccion del Modelo
    y_preds_forest=random_forest.predict(X_test_trans)

    # Convertir el DataFrame a un archivo CSV
    nombre_archivo = 'predicciones.csv'
    
    with open(nombre_archivo, 'w', newline='') as archivo_csv:
        escritor = csv.writer(archivo_csv)
        escritor.writerow(y_preds_forest)

    

In [24]:
print("¿Qué desea hacer?")
print("1. Entrenar Modelo")
print("2. Obtener Predcciones del Modelo")
print("3. Salir de la Transaccion")

¿Qué desea hacer?
1. Entrenar Modelo
2. Obtener Predcciones del Modelo
3. Salir de la Transaccion


In [25]:

## Definicion de variables iniciales
transaccion=0
fin=3

## Ciclo que continua mientras no se ingrese la opcion 3-Terminar Transaccion
while (True):

    transaccion=int(input("Ingrese el número de la transacción:"))
    

    ## Valida si se ingresa un numero entre 1 y 3, sino devuleve mensaje de transaccion invalida
    if transaccion>=1 and transaccion<=3:
        ## Mientras no se ingrese 3, se corre el programa
        if(fin!=transaccion):
                                              
                ## Si se ingresa 1, corre funcion entrenar modelo
                if transaccion==1:
                    train_model()
                     
                ## Si se ingresa 2, corre funcion prediccion modelo          
                else: 
                     predict()
    
        else:
            break
    else:
         print("Transaccion invalida")




